In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from pydantic import BaseModel, Field

class Evaluation(BaseModel):
  "Evaluation metric for history taking field"

  score: int = Field(description="If the relevant information is obtained by the doctor in the history taking conversation, give a score of 1. Else, 0.")
  explanation: str = Field(description="Explain in 1 sentence why the score was given.")


In [11]:
from langchain_core.prompts import PromptTemplate

conversation = """
Doctor: What brings you in today? | Patient: I have an ongoing chest pain for the last 1 day.
Doctor: Can you tell me more about character of the chest pain? | Patient: It is a persistent dull pain.
"""

history_taking_category = "Radiation"
patient_response = "Pain radiates to the neck, upper back and left arm."

template = """
You are an expert evaluator of doctors taking history from a patient.

This is the conversation between the doctor and the patient:
{conversation}

Now make an assessment if the doctor asked questions pertaining to "{history_taking_category}" to get the following "{patient_response}"
"""

prompt = PromptTemplate.from_template(template)

structured_llm = llm.with_structured_output(Evaluation)

chain = prompt | structured_llm

output = chain.invoke({"conversation": conversation, "history_taking_category": history_taking_category, "patient_response": patient_response})


In [14]:
output.score

0

In [15]:
output.explanation

"The doctor did not ask about the radiation of the chest pain, which is critical for understanding the pain's characteristics."

In [38]:
from importlib import reload
import simulator
reload(simulator)
from simulator import ExaminerAgent


In [39]:
simulatedExaminer = ExaminerAgent(model_name="gpt-4o-mini")

output = simulatedExaminer.evaluate_history_taking_item(conversation, history_taking_category, patient_response)

In [46]:
physical_exam_suggestions = """
1. **Vital Signs**: I'll check your pulse rate, blood pressure, respiratory rate, and oxygen saturation using my stethoscope.
2. **Percussion**: I'll gently tap on your chest with my fingers to assess the sound of your heart and lungs.
3. **Palpation**: I'll use my hands to feel for any abnormalities in your chest wall, such as tenderness or masses.
4. **Auscultation**: I'll listen carefully with my stethoscope to hear the sounds of your heart and lungs, looking for any irregularities or murmurs.
"""

physical_exam_category = "Neck Exam"
physical_exam_details = "Looked for JVD, carotid auscultation"


In [60]:
from importlib import reload
import simulator
reload(simulator)
from simulator import ExaminerAgent

simulatedExaminer = ExaminerAgent(model_name="gpt-4o-mini")


In [48]:
output = simulatedExaminer.evaluate_physical_exam_item(
  physical_exam_suggestions, physical_exam_category, physical_exam_details
)

In [50]:
output.explanation

'The doctor did not suggest any steps related to the neck examination, such as checking for jugular venous distension (JVD) or performing carotid auscultation.'

In [51]:
import pandas as pd

physical = pd.read_excel("Data/Case_1.xlsx", sheet_name="Physical Examination")


In [61]:
simulatedExaminer.evaluate_physical_exam_checklist(physical_exam_suggestions, physical.loc[:5])

In [62]:
simulatedExaminer.physical_exam_score

2

In [70]:
from importlib import reload
import simulator
reload(simulator)
from simulator import ExaminerAgent

simulatedExaminer = ExaminerAgent(model_name="gpt-4o-mini")


In [68]:
investigations_suggestions = """
1. **Electrocardiogram (ECG or EKG)**: This is a non-invasive test that measures the electrical activity of the heart. It can help identify any irregularities in the heartbeat, which could indicate a cardiac issue.
2. **Blood tests**: I'd like to run some blood tests to check your:
	* Complete Blood Count (CBC) to rule out anemia or infection
	* Electrolyte levels (e.g., potassium, sodium) to ensure they're within normal ranges
	* Lipid profile to assess your cholesterol and triglyceride levels
	* Troponin test to check for cardiac damage
3. **Chest X-ray**: This will help me evaluate the size and shape of your heart, as well as any potential lung issues.
4. **Echocardiogram (Echo)**: This non-invasive ultrasound test will allow me to visualize the structure and function of your heart, including the valves and chambers.
5. **Stress Test**: Depending on the results of the above tests, I may recommend a stress test to assess how your heart responds to physical activity.
"""

investigations_item = "ECG"

In [69]:
simulatedExaminer.evaluate_investigations_item(investigations_suggestions, investigations_item)

InvestigationsEvaluation(score=1, explanation='The doctor suggested an Electrocardiogram (ECG), which is appropriate for assessing irregularities in the heartbeat and potential cardiac issues.')

In [71]:
import pandas as pd

investigations = pd.read_excel("Data/Case_1.xlsx", sheet_name="Investigations")

simulatedExaminer.evaluate_investigations_checklist(investigations_suggestions, investigations)


In [72]:
simulatedExaminer.investigations_score

3

In [73]:
simulatedExaminer.investigations_scoring_log

['The doctor suggested an Electrocardiogram (ECG), which is appropriate for assessing irregularities in the heartbeat and potential cardiac issues.',
 'The doctor suggested a troponin test, which is a key cardiac enzyme used to assess for cardiac damage, making it appropriate for evaluating cardiac issues.',
 'The doctor suggested an echocardiogram, which is a type of transthoracic echocardiography, to visualize the structure and function of the heart.',
 'The suggested investigations focus on cardiac health and do not address any gastrointestinal issues that would warrant an upper endoscopy.',
 'The suggested investigations focus on cardiac health and general blood analysis, which do not address the need for a urine toxicology test.']

In [84]:
from importlib import reload
reload(simulator)
from simulator.agents import ExaminerAgent

simulatedExaminer = ExaminerAgent(model_name="gpt-4o-mini")


In [85]:
ddx_suggestions = """
1. **Acute Coronary Syndrome (ACS)**: Given the sudden onset of chest pain, shortness of breath, sweating, and nausea, ACS is a strong possibility. This condition can be caused by a blockage in one or more coronary arteries, leading to reduced blood flow to the heart muscle.
2. **Pulmonary Embolism (PE)**: Joseph's symptoms of sudden onset chest pain, shortness of breath, and sweating could also suggest a pulmonary embolism. A PE occurs when a blood clot lodges in an artery in the lung, blocking blood flow and causing these symptoms.
3. **Acid Reflux or Gastroesophageal Reflux Disease (GERD)**: Although Joseph has a history of GERD, it's possible that his current symptoms could be related to acid reflux or GERD exacerbation. However, given the severity of his symptoms, I would need to rule out other more serious conditions first.
"""
ddx_item = "Myocardial Infarction"

simulatedExaminer.evaluate_ddx_item(ddx_suggestions, ddx_item)

DdxEvaluation(score=1, explanation='The doctor suggested Acute Coronary Syndrome (ACS), which includes Myocardial Infarction as a potential cause of the symptoms, making it an appropriate differential diagnosis.')

In [78]:
import pandas as pd

diagnosis = pd.read_excel("Data/Case_1.xlsx", sheet_name="Differential Diagnosis")

# diagnosis

simulatedExaminer.evaluate_ddx_checklist(ddx_suggestions, diagnosis)

In [80]:
simulatedExaminer.differential_diagnoses_score

2

In [81]:
simulatedExaminer.differential_diagnoses_scoring_log

['The doctor suggested Acute Coronary Syndrome (ACS), which includes myocardial ischemia or infarction as a key component, making it an appropriate differential diagnosis.',
 "The doctor did not suggest Cocaine induced myocardial ischemia, which is a relevant differential diagnosis given the patient's symptoms and potential drug use.",
 "The doctor appropriately included GERD as a differential diagnosis due to the patient's history, while also recognizing the need to prioritize more serious conditions given the severity of the current symptoms."]